In [2]:
import os
os.chdir("../")
%pwd

'c:\\Codes\\Codes\\Python\\Summlytics'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    FILE_STATUS: str
    ALL_REQUIRED_FILES: list

In [31]:
from Summlytics.constants import *
from Summlytics.utils.common import read_yaml, create_directories

In [32]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:

        config = self.config.data_validation

        create_directories([config.root_dir])
        
        return DataValidationConfig(
            root_dir = config.root_dir,
            FILE_STATUS = config.FILE_STATUS,
            ALL_REQUIRED_FILES = config.ALL_REQUIRED_FILES
        )

In [38]:
from Summlytics.logging import logger
import json

In [39]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    # def validate_all_files(self):
    #     """Validate all files in the data directory"""
    #     # return all([validate_file() for validate_file in [self.validate_file(file) for file in self.config.ALL_REQUIRED_FILES]])
    #     return all(self.validate_file(file) for file in self.config.ALL_REQUIRED_FILES)
    
    # def validate_file(self, file: str) -> bool:
    #     """Validate a file in the data directory"""
    #     file_path = Path(self.config.root_dir + "/" + file)
    #     if not file_path.exists():
    #         logger.error(f"Missing file: {file_path}")
    #         return False
    #     logger.info(f"File {file_path} exists")
    #     return True
        
    # def validate_all_files(self):
    #     """Validate all files in the data directory"""
    #     try:
    #         validation_status = None

    #         file_loc = os.listdir(os.path.join("artifacts", "data_ingestion", "samsum_dataset"))

    #         for file in file_loc:
    #             if file not in self.config.ALL_REQUIRED_FILES:
    #                 validation_status = False
    #                 with open(self.config.FILE_STATUS, "w") as f:
    #                     f.write(f"Validation status: {validation_status}")
    #             else:
    #                 validation_status = True
    #                 with open(self.config.FILE_STATUS, "w") as f:
    #                     f.write(f"Validation status: {validation_status}")

    #         return validation_status
        
    #     except Exception as e:
    #         logger.error(f"Validation failed: {e}")
    #         return False
    
    def validate_all_files(self) -> bool:
        try:
            all_valid = all(file in self.config.ALL_REQUIRED_FILES for file in os.listdir(os.path.join("artifacts", "data_ingestion", "samsum_dataset")))
            with open(self.config.FILE_STATUS, "w") as f:
                json.dump({"validation_status": all_valid}, f)
            return all_valid
        except Exception as e:
            logger.error(f"Validation failed: {e}")
            return False

In [41]:
try:
    logger.info("Data Validation started")

    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(data_validation_config)
    data_validation.validate_all_files()

    logger.info("Data Validation completed")
except Exception as e:
    logger.error(e)
    raise e

[2024-02-07 19:23:47,724: INFO: 547802505: Data Validation started]
[2024-02-07 19:23:47,725: INFO: common: yaml file: <_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='cp1252'> loaded successfully.]
[2024-02-07 19:23:47,726: INFO: common: yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='cp1252'> loaded successfully.]
[2024-02-07 19:23:47,727: INFO: common: Directory: artifacts already exists.]
[2024-02-07 19:23:47,728: INFO: common: Directory: artifacts/data_validation created.]
False
[2024-02-07 19:23:47,729: INFO: 547802505: Data Validation completed]
